In [9]:
'''The goal of this code base is to create a backend database for an inventory managemetn system. 
Which consist of a Products, Orders, Suppliers, OrderDetals, and Transactions

Products Table:
Prod_ID VARCHAR (20)
Prod_Name VARCHAR (50)
Description REAL
Price FLOAT
Quantity_Aval INT

Suppliers Table:
SupplierID VARCHAR (20) (Primary Key)
SupplierName VARCHAR (50)
ContactInfo REAL
Address REAL

Orders Table:
OrderID VARCHAR (20) (Primary Key),
OrderDate DATE,
SupplierID VARCHAR (20) (Foreign Key referencing the Suppliers table),
TotalAmount INT

OrderDetails Table:
OrderDetailID VARCHAR (20) (Primary Key),
OrderID VARCHAR (20) (Foreign Key referencing the Orders table),
ProductID VARCHAR(20) (Foreign Key referencing the Products table),
QuantityOrdered INT,
UnitPrice FLOAT,

Transactions Table:
TransactionID VARCHAR (20)(Primary Key),
TransactionDate DATE,
ProductID VARCHAR (20) (Foreign Key referencing the Products table),
Quantity INT,
TransactionType VARCHAR (25),
Remarks TEXT,
'''




'The goal of this code base is to create a backend database for an inventory managemetn system. \nWhich consist of a Products, Orders, Suppliers, OrderDetals, and Transactions\n\nProducts Table:\nProd_ID VARCHAR (20)\nProd_Name VARCHAR (50)\nDescription REAL\nPrice FLOAT\nQuantity_Aval INT\n\nSuppliers Table:\nSupplierID VARCHAR (20) (Primary Key)\nSupplierName VARCHAR (50)\nContactInfo REAL\nAddress REAL\n\nOrders Table:\nOrderID VARCHAR (20) (Primary Key),\nOrderDate DATE,\nSupplierID VARCHAR (20) (Foreign Key referencing the Suppliers table),\nTotalAmount INT\n\nOrderDetails Table:\nOrderDetailID VARCHAR (20) (Primary Key),\nOrderID VARCHAR (20) (Foreign Key referencing the Orders table),\nProductID VARCHAR(20) (Foreign Key referencing the Products table),\nQuantityOrdered INT,\nUnitPrice FLOAT,\n\nTransactions Table:\nTransactionID VARCHAR (20)(Primary Key),\nTransactionDate DATE,\nProductID VARCHAR (20) (Foreign Key referencing the Products table),\nQuantity INT,\nTransactionType 

In [10]:
import glob
import os
import pandas as pd 
import psycopg2
from sqlalchemy import create_engine
import sqlite3


In [11]:
# Establish Postgres connection
connection_params = {
    'user': 'username',
    'password': 'password',
    'host': 'host',
    'port': '5000',
    'database': 'IMS_db'
}

In [12]:
db_name = 'IMS.db'
engine = create_engine('sqlite:///'+ db_name, echo=False)
file_dir = '/Users/rashaanrichardson/Desktop/SQL/Inventory_Mgt_Sys'
csv_files = '/Users/rashaanrichardson/Desktop/SQL/Inventory_Mgt_Sys/IMS_CSV'

conn = sqlite3.connect(db_name)
c = conn.cursor()

In [13]:
# c.execute('''DROP TABLE Product''')
# c.execute('''DROP TABLE Orders''')
# c.execute('''DROP TABLE Suppliers''')
# c.execute('''DROP TABLE OrderDetails ''')
# c.execute('''DROP TABLE Transactions''')
tables = c.fetchall()
for table in tables:
        print(table[0])

In [14]:
# Create Tables
# Products 
c.execute('''CREATE TABLE IF NOT EXISTS Products(
        ProdID VARCHAR (20) PRIMARY KEY NOT NULL,
        ProdName VARCHAR (50) NOT NULL,
        Description REAL,
        Price FLOAT,
        QuantityAval INT  
)''')
# Suppliers
c.execute(''' CREATE TABLE IF NOT EXISTS Suppliers(
        SupplierID VARCHAR (20) PRIMARY KEY NOT NULL,
        SupplierName VARCHAR (50) NOT NULL,
        ContactInfo REAL,
        Address REAL
)''')
# Orders
c.execute('''CREATE TABLE IF NOT EXISTS Orders(
        OrderID VARCHAR (20) PRIMARY KEY NOT NULL,
        OrderDate DATE,
        SupplierID VARCHAR(20) NOT NULL,
        TotalAmount INT,
        FOREIGN KEY(SupplierID) REFERENCES Suppliers(SupplierID)
)''')
# Order Details
c.execute(''' CREATE TABLE IF NOT EXISTS OrderDetails(
        OrderDetailID VARCHAR (20) PRIMARY KEY NOT NULL,
        OrderID VARCHAR (20)NOT NULL,
        ProductID VARCHAR (20),
        QuantityOrdered INT,
        UnitPrice FLOAT,
        FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
        FOREIGN KEY (ProductID) REFERENCES Products(ProdID)
)''')
# Transactions
c.execute('''CREATE TABLE IF NOT EXISTS Transactions(
        TransactionID VARCHAR (20) PRIMARY KEY NOT NULL,
        TransactionDate DATE,
        ProdID VARCHAR (20),
        Quantity INT,
        TransactionType VARCHAR (25),
        Remarks TEXT,
        FOREIGN KEY (ProdID) REFERENCES Products(ProdID)
)''')

# Fetch table names from SQLite database
c.execute('''SELECT name FROM sqlite_master WHERE type='table';''')
tables = c.fetchall()
all_tables = [table[0] for table in tables]  # Extract table names from fetched data
print(all_tables)

['OrderDetails', 'Transactions', 'Orders', 'Suppliers', 'Products']


In [15]:
# Iterate over CSV files in the directory
for file in glob.glob(os.path.join(csv_files, '*.csv')):
    # Isolate the name
    insert_name = os.path.basename(file).removesuffix('_Insert.csv')
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file)
        # Check if the table exists in the database
        if insert_name in all_tables:
            print('Is in',insert_name,all_tables)
            # If the table exists, replace its contents with data from the CSV file
            df.to_sql(insert_name, con=engine, if_exists='replace', index=False)
    #         # Print the contents of the table
            query = f"SELECT * FROM {insert_name}"
            print(f"Contents of table '{insert_name}':")
            print(pd.read_sql(query, con=engine))
        else:
            print(f"Table {insert_name} does not exist in the database.")
    except Exception as e:
        print(f"Error processing file {file}: {e}")




Is in OrderDetails ['OrderDetails', 'Transactions', 'Orders', 'Suppliers', 'Products']
Contents of table 'OrderDetails':
   OrderDetailID  OrderID  ProductID  QuantityOrdered  UnitPrice
0              1        1          1               10      10.99
1              2        1          2                5      29.99
2              3        2          3                8      19.99
3              4        2          4                3      39.99
4              5        3          5               15      49.99
Is in Transactions ['OrderDetails', 'Transactions', 'Orders', 'Suppliers', 'Products']
Contents of table 'Transactions':
   TransactionID TransactionDate  ProdID  Quantity TransactionType  \
0              1      2024-02-25       1        20         Inbound   
1              2      2024-02-26       2        10        Outbound   
2              3      2024-02-27       3         5         Inbound   
3              4      2024-02-28       4         2        Outbound   
4              5  